<a href="https://colab.research.google.com/github/hansubkim808/spotify_net/blob/main/CS230_Project_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spotipy

# **Setup**

In [ ]:
import os

os.environ['SPOTIPY_CLIENT_ID']='a78e6d0e2ed240d2813c130a3e848194'
os.environ['SPOTIPY_CLIENT_SECRET']='74ceb730c73b41a98fbfdb4bda9a121d'
os.environ['SPOTIPY_REDIRECT_URI']='https://developer.spotify.com/dashboard/applications/a78e6d0e2ed240d2813c130a3e848194'

In [ ]:
import spotipy
import sys
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import Model
from numpy import linalg as LA
import pandas as pd 
from statistics import mean, mode 

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())


# **Helper Functions**

**get_artist_ids:**
This function retrieves spotify artist IDs from a list of artists defined by the user

In [ ]:
def get_artist_ids(artists):

  artist_ids = []
  artist_names = []

  for artist in artists:
    artist_results = sp.search(artist, type='artist')
    artist_name = artist_results['artists']['items'][0]['name']
    artist_id = artist_results['artists']['items'][0]['id']
    if artist_name in artist_names:
      continue
    else:
      artist_names.append(artist_name)
      artist_ids.append(artist_id)

  return artist_ids, artist_names

**get_album_ids:**
This function creates a list of spotify album IDs from a list of artist IDs passed into the function

In [ ]:
def get_album_ids(artist_ids):

  album_ids = []
  album_names = []

  for artist_id in artist_ids:
    album_results = sp.artist_albums(artist_id, limit=50)
    for album in album_results['items']:
      album_id = album['id']
      album_name = album['name']
      if album_name in album_names:
        continue
      else:
        album_names.append(album_name)
        album_ids.append(album_id)

  return album_ids, album_names

**get_track_ids:**
This function createds a list of spotify track IDs from a list of album IDs passed into the function

In [ ]:
def get_track_ids(album_ids):

  track_ids = []
  track_names = []

  for album_id in album_ids:
    track_results = sp.album_tracks(album_id, limit=50)
    for track in track_results['items']:
      track_id = track['id']
      track_name = track['name']
      if track_name in track_names:
        continue
      else:
        track_names.append(track_id)
        track_ids.append(track_name)

  return track_ids, track_names

**get_tracks_from_artist_ids:**
    Given a list of artist ids, creates a master list of every single track
    from all the artists, as well as the track ids. Combines all 3 functions above. Working example code available in the other colab notebook


In [ ]:
def get_tracks_from_artist_ids(artist_ids): # takes in a list of artists 

  all_album_ids = []
  all_album_titles = []

  # Grab all albums from each artist
  for id in artist_ids:
    albums = sp.artist_albums(id)
    album_titles = [albums['items'][i]['name'] for i in range(len(albums))]
    album_ids = [albums['items'][i]['id'] for i in range(len(albums))]
    all_album_ids.extend(album_ids)
    all_album_titles.extend(album_titles)

  all_track_ids = []
  all_track_names = []

  # Grab all tracks in each album
  for i in range(len(all_album_ids)):
    album = all_album_ids[i]
    album_tracks = sp.album_tracks(album)
    album_track_ids = [album_tracks['items'][j]['id'] for j in range(album_tracks['total'])]
    album_track_names = [album_tracks['items'][k]['name'] for k in range(album_tracks['total'])]
    all_track_ids.extend(album_track_ids)
    all_track_names.extend(album_track_names)
  
  # Remove duplicates 
  tracks_dict = {all_track_ids[i] : all_track_names[i] for i in range(len(all_track_ids))}
  temp = {val : key for key, val in tracks_dict.items()} 
  res = {val : key for key, val in temp.items()} 
  total_track_ids = list(res.keys())
  total_track_names = list(res.values())

  return all_album_titles, all_album_ids, total_track_names, total_track_ids


In [ ]:
"""
    Returns the artist, given track ID. 

    This adds 1 parameter to the existing 0

    TOTAL: 1
"""

# INCOMPLETE 


def get_artists(track_ids):
  return artists_df


**get_sp_audio_features:** Returns 13 audio feature parameters for each track_id in the master list of track_ids. This adds 13 parameters to the existing 1 (Total: 14) 

In [ ]:
def get_sp_audio_features(track_ids):
  audio_features = [sp.audio_features(id) for id in track_ids]
  feature_dicts = [elem[0] for elem in audio_features] # remove list wrapper, leave dict intact
  audio_feature_df = pd.DataFrame(feature_dicts) # organize into a table 
  audio_feature_df = audio_feature_df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url'], axis=1) # remove irrelevant columns

  return audio_feature_df

**get_analysis_segments:** Spotify audio analysis (from the API) contains important mathematical info about the track in the "segments" section. This function isolates the "segments" section from the rest of the audio_analysis beacuse it is computationally expensive to calculate in a loop. 

In [ ]:
def get_analysis_segments(track_ids):
  audio_analysis = [sp.audio_analysis(id) for id in track_ids] # get audio analysis for each track in list of tracks
  track_segments = [audio_analysis[i]['segments'] for i in range(len(audio_analysis))] # get 'segments' portion of audio analysis for each track

  return track_segments


**get_pitches:** Given a list of segments (each track id corresponds to 1 segment): Extracts a dictionary corresponding to the relative prominence of the pitch of each note (from C to C). A dataframe is formed from this list of dicts, to be appended later to the other parameter dataframes. This adds 12 parameters to the existing 14. (Total: 26)


In [ ]:
# INCOMPLETE 

def get_pitches(track_segments):
  note_index = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

  all_pitches = []

  for i in range(len(track_segments)):
    segment = track_segments[i]
    pitches = [segment[j]['pitches'] for j in range(len(segment))]
    chroma_vector = []
    for k in range(len(pitches[0])):
      chroma_vector.append(mean(pitch_vec[k] for pitch_vec in pitches))
    chroma_dict = {note_index[n]: chroma_vector[n] for n in range(len(note_index))}  
    dominant_overall_pitch = sorted(chroma_vector)[-1]
    #top_pitch_note = chroma_dict[dominant_overall_pitch]
    #top_note_dict[top_pitch_note] = dominant_overall_pitch
    all_pitches.append(chroma_dict)

  pitches_df = pd.DataFrame(all_pitches)

  return pitches_df

**get_timbres:** Given a list of segmgnets (each track id corresponds to 1 segment): Extracts a dictionary corresponding to the relative prominence of each timbre quality (1 thru 12). A dataframe is formed from this list of dicts, to be appended later to the other parameter dataframes. This adds 12 parameters to the existing 26 (Total: 38) 

In [ ]:
# INCOMPLETE 

def get_timbres(track_segments):

  all_timbres = []
  timbre_labels = ['Loudness', 'Brightness', 'Flatness', 'Strength of Attack', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12']
  for i in range(len(track_segments)):
    segment = track_segments[i]
    timbres = [segment[j]['timbre'] for j in range(len(segment))]
    overall_timbre = []
    for k in range(len(timbres[0])):
      overall_timbre.append(mean(timbre_vec[k] for timbre_vec in timbres))
    timbre_dict = {timbre_labels[i] : overall_timbre[i] for i in range(len(timbre_labels))}
    all_timbres.append(timbre_dict)
  timbres_df = pd.DataFrame(all_timbres)

  return timbres_df

# **Preprocessing**

In [ ]:
# INCOMPLETE

def get_input_features(track_ids)
  artists_df = get_artist(track_ids)
  audio_feature_df = get_sp_audio_features(track_ids)
  track_segments = get_analysis_segments(track_ids)
  pitches_df = get_pitches(track_segments)
  timbres_df = get_timbres(track_segments)
  full_input_features_df = pd.concat([artists_df, audio_feature_df, pitches_df, timbres_df], axis=1)
  return full_input_features_df

SyntaxError: ignored

In [ ]:
# INCOMPLETE
# Returns a list of labels given a list of track ids
def get_labels(track_ids)
  return labels

In [ ]:
# COMPLETE
# Formalizes dataset
def get_dataset(track_ids)
  X = get_input_features(track_ids)
  Y = get_labels(track_ids)
  return X,Y

In [ ]:
# Splits up train, test, val from X,y, returns dictionary of all
def split_train_and_test (X,Y)
  train_size = len(x)*7/10
  #split em up:
  x_train = 
  y_train = 
  x_test = 
  y_test = 
  x_val =
  y_val =

  data_dict =  {
  "X": X,
  "Y": Y,
  "x_train": x_train,
  "y_train": y_train,
  "x_test": x_test,
  "y_test": y_test,
  "x_val" : x_val,
  "y_val": y_val
  }
  return data_dict

# **Deep Learning**

**Get list of track IDs to extract data from for input**

In [ ]:
# List of 50 most popular artists and 50 most underrated artists.
artists = ['Post Malone', 'Duke Deuce', 'Drake', 'Lil Nas X', 'Travis Scott', 'Juice WRLD', 'DaBaby', 'Cardi B',
           'Lil Baby', 'Meek Mill', 'A Boogie Wit da Hoodie', 'Tee Grizzley', '21 Savage', 'SOB X RBE',
           'Chris Brown', 'Kodak Black', 'J. Cole', 'Young Thug', 'Lil Tecca', 'YoungBoy Never Broke Again',
           'YNW Melly', '9lokknine', 'Swae Lee', 'Eminem', 'NF', 'DJ Khaled', 'Offset', 'Lil Wayne',
           'Tyler, The Creator', 'Future', 'Blueface', 'Gunna', 'Nipsey Hussle', 'Mustard',
           'Megan Thee Stallion', 'Lil Uzi Vert', 'Polo G', '6ix9ine', 'City Girls', 'Trippie Redd',
           'Kanye West', 'Sheck Wes', 'Gucci Mane', 'Doe Boy', 'Migos', 'Roddy Ricch', 'Calboy',
           'Nicki Minaj', 'Kendrick Lamar', 'King Von', 'Jackboy', 'Shootergang Kony', '1takejay',
           'Lil Durk', 'Lil Keed', 'Lil Gotit', 'OMB Peezy', 'Foogiano', 'Big Scarr',
           'Action Pack', 'Pooh Shiesty', 'Pop Smoke', 'Fivio Foreign', '22Gz', 'K Shiday', 'Mulatto',
           '24kGoldn', 'Rich Homie Quan', 'Key Glock', 'Young Dolph', 'ABG Neal', '42 Dugg',
           'Cash Kidd', 'Bfb Da Packman', 'Teejayx6', 'Kasher Quon', 'Sada Baby', 'Drakeo the Ruler',
           'AzChike', '$tupid Young', 'Pacman da Gunman', 'Rio Da Yung Og', 'BlueBucksClan', 'Lil Zay Osama', 'Lil Poppa',
           '600 Breezy', 'Sheff G', 'Sleepy Hallow', 'Yungeen Ace', 'JayDaYoungan', 'Playboi Carti', 'UnoTheActivist',
           'Thouxanbanfauni', 'Blocboy JB', 'NLE Choppa', 'Rod Wave', 'Quando Rondo', 'Slatt Zy']

artist_ids, artist_names = get_artist_ids(artists)
album_ids, album_names = get_album_ids(artist_ids)
track_ids, track_names = get_track_ids(album_ids)
print(len(track_ids))

24553


In [ ]:
# TODO: build the specified model, see its summary
in_size = X.shape[1]
out_size = X.shape[1]
batch_size = 32
hidden_layer_size = batch_size
model = keras.Sequential()
model.add(keras.layers.Dense(hidden_layer_size,input_dim=in_size,activation='relu',kernel_initializer='he_normal'))
model.add(keras.layers.Dense(out_size,input_dim=hidden_layer_size,kernel_initializer='he_normal'))
model.summary()

NameError: ignored

In [ ]:
# TODO: Train the model, feel free to insert more cells
model.compile(optimizer=tf.keras.optimizers.Adam(.01), loss = 'mean_squared_error')
history = model.fit(x,y,batch_size=batch_size,epochs = 100, verbose = 0)
plt.plot(history.history['loss'], label='training')
plt.title('Training Loss')
plt.ylabel('MSE loss')
plt.xlabel('No. epoch')
plt.legend()
plt.show()